In [10]:
import numpy as np
import pandas as pd
from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.table import WD_ALIGN_VERTICAL

In [11]:
# # 加载 Word 文档
# doc = Document('应收询证函模板.docx')  # 将 'example.docx' 更改为你的文件名

# # 遍历文档的每个节
# for section in doc.sections:
#     # 获取每个节的页眉
#     header = section.header
#     # 遍历页眉中的每一个段落
#     for para in header.paragraphs:
#         # 遍历段落中的每一个 run
#         for run in para.runs:
#             # 从 run 中找到所有的 drawing 元素（浮动图片通常嵌入在 drawing 元素中）
#             drawings = run._r.findall('.//' + qn('w:drawing'))
#             for drawing in drawings:
#                 # 将 drawing 元素替换为一个空的元素，从而删除图片
#                 run._r.replace(drawing, parse_xml('<w:r xmlns:w="http://schemas.openxmlformats.org/wordprocessingml/2006/main"></w:r>'))

# # 保存文档
# doc.save('example_no_header_images.docx')  # 将 'example_no_header_images.docx' 更改为你的新文件名




In [12]:
# 读入dataframe 计算表格行数
df = pd.read_excel('../应收票据-函证-明细表.xlsx', sheet_name='2应收票据', usecols=[i for i in range(20)])
df.shape

(373, 18)

In [13]:
# 2000-01-01
# df['出票日'] = df['出票日'].apply(lambda x: x.strftime(format='%Y-%m-%d'))
# df['截止日期'] = df['截止日期'].apply(lambda x: x.strftime(format='%Y-%m-%d'))
# df['到期日'] = df['到期日'].apply(lambda x: x.strftime(format='%Y-%m-%d'))

In [14]:
# 2000年1月1日
df['出票日'] = df['出票日'].apply(lambda x: x.strftime(format="%Y年%m月%d日").replace('年0', '年').replace('月0', '月'))
df['截止日期'] = df['截止日期'].apply(lambda x: x.strftime(format="%Y年%m月%d日").replace('年0', '年').replace('月0', '月'))
df['到期日'] = df['到期日'].apply(lambda x: x.strftime(format="%Y年%m月%d日").replace('年0', '年').replace('月0', '月'))

In [15]:
comp_number = df.groupby('被询证单位')['询证函编号'].count()

In [16]:
comp_number

被询证单位
上海瀚讯信息技术股份有限公司       1
上海航天电子通讯设备研究所        2
上海航空电器有限公司           1
中兵通信科技股份有限公司         5
中国兵器工业计算机应用技术研究所     5
                    ..
长沙湘计海盾科技有限公司        10
陕西中科天地航空模块有限公司      14
陕西铭贤电子科技有限公司        10
陕西雷能电子科技有限公司         6
青岛航天半导体研究所有限公司      41
Name: 询证函编号, Length: 62, dtype: int64

In [17]:
heji = df.groupby(['被询证单位','年份'])['金额'].sum()

In [18]:
heji

被询证单位           年份  
上海瀚讯信息技术股份有限公司  2022    1000000.0
上海航天电子通讯设备研究所   2021     176000.0
                2022      79000.0
上海航空电器有限公司      2023      48002.4
中兵通信科技股份有限公司    2021     500000.0
                          ...    
陕西雷能电子科技有限公司    2022     697400.0
青岛航天半导体研究所有限公司  2020    2116554.0
                2021    2079870.0
                2022    5591900.0
                2023    2402400.0
Name: 金额, Length: 130, dtype: float64

In [19]:
# 替换单位名称
def docx_find_replace_text(doc, search_text, replace_text):
    for paragraph in doc.paragraphs:
        if search_text in paragraph.text:
            for run in paragraph.runs:
                if search_text in run.text:
                    run.text = run.text.replace(search_text, replace_text)

# docx_find_replace_text(doc, '«name»', 'new text')

In [20]:
# # 替换编号
# def docx_find_replace_header(doc, search_text, replace_text):
#     for section in doc.sections:
#         header = section.header  # 获取节的页眉
#         for paragraph in header.paragraphs:  # 遍历页眉的每个段落
#             if search_text in paragraph.text:
#                 for run in paragraph.runs:
#                     if search_text in run.text:
#                         run.text = run.text.replace(search_text, replace_text)

# # docx_find_replace_header(doc, '«c_id»', 'new text')

In [21]:
def set_cell_text_style(cell, pos): #确定样式
    cell.vertical_alignment = WD_ALIGN_VERTICAL.CENTER # 垂直居中
    for para in cell.paragraphs:
        if pos.lower() == "left":
            para.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
        elif pos.lower() == "mid":
            para.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
        elif pos.lower() == "right":
            para.alignment = WD_PARAGRAPH_ALIGNMENT.RIGHT
        else:
            raise ValueError("Invalid position value. Please use 'left', 'mid' or 'right'.")
        for run in para.runs:
            run.font.size = Pt(10.5) # 五号字体
            run.font.name = u"宋体"
            run._element.rPr.rFonts.set(qn('w:eastAsia'), u'宋体')
    return cell

In [22]:
# 根据每个公司不同的票据数，生成不同的行数
for name in comp_number.index:
    doc = Document('./example_应收票据.docx')
    c_id = df[df['被询证单位']==name]['询证函编号'].drop_duplicates().values[0]
    docx_find_replace_text(doc, '«c_id»', c_id)
    docx_find_replace_text(doc, '«name»', name)
    table_num = 0
    
    for year in [2020,2021,2022,2023]:
        temp = df[(df['被询证单位']==name) & (df['年份']==year)]
        r_num, c_num = temp.shape
        
        if r_num==0: # 当年无记录
            table_num += 1
            
        elif r_num == 1:  # 当年只有一条记录 
            r = doc.tables[table_num].rows[1]
            r.cells[0].text = '1' ; set_cell_text_style(r.cells[0], 'mid') # 序号
            r.cells[1].text = temp.iloc[0, 4].strip() ; set_cell_text_style(r.cells[1], 'left') # 票据种类
            r.cells[2].text = temp.iloc[0, 5].strip() ; set_cell_text_style(r.cells[2], 'left') # 出票单位名称
            r.cells[3].text = temp.iloc[0, 6].strip() ; set_cell_text_style(r.cells[3], 'left') # 承兑人
            r.cells[4].text = temp.iloc[0, 7] ; set_cell_text_style(r.cells[4], 'left') # 出票日
            r.cells[5].text = temp.iloc[0, 8] ; set_cell_text_style(r.cells[5], 'left') # 到期日
            r.cells[6].text = temp.iloc[0, 9] ; set_cell_text_style(r.cells[6], 'left') # 票号
            r.cells[7].text = "{:,.2f}".format(temp.iloc[0, 10]) ; set_cell_text_style(r.cells[7], 'right') # 金额
            
            # 书写完成后删除行
            # row = doc.tables[table_num].rows[1]._element
            # row.getparent().remove(row)
            table_num += 1
        else:   
            for i in range(r_num):
                adr = doc.tables[table_num].add_row()
                adr.cells[0].text = str(i+1) ; set_cell_text_style(adr.cells[0], 'mid')# 序号
                adr.cells[1].text = temp.iloc[i, 4].strip() ; set_cell_text_style(adr.cells[1], 'left')# 票据种类
                adr.cells[2].text = temp.iloc[i, 5].strip() ; set_cell_text_style(adr.cells[2], 'left')# 出票单位名称
                adr.cells[3].text = temp.iloc[i, 6].strip() ; set_cell_text_style(adr.cells[3], 'left')# 承兑人
                adr.cells[4].text = temp.iloc[i, 7] ; set_cell_text_style(adr.cells[4], 'left')# 出票日
                adr.cells[5].text = temp.iloc[i, 8] ; set_cell_text_style(adr.cells[5], 'left')# 到期日
                adr.cells[6].text = temp.iloc[i, 9] ; set_cell_text_style(adr.cells[6], 'left')# 票号
                adr.cells[7].text = "{:,.2f}".format(temp.iloc[i, 10]) ; set_cell_text_style(adr.cells[7], 'right')# 金额
                
            # 循环结束后删除第二行
            row = doc.tables[table_num].rows[1]._element
            row.getparent().remove(row)
            
            # 循环结束后增加合计部分
            row_cells = doc.tables[table_num].add_row().cells
            merged_cell = row_cells[0].merge(row_cells[1]).merge(row_cells[2]).merge(row_cells[3]).merge(row_cells[4])\
                .merge(row_cells[5]).merge(row_cells[6])

            # 将合并后的单元格的文本设置为"合计"
            merged_cell.text = '合计:' ; set_cell_text_style(merged_cell, 'mid')
            row_cells[7].text = "{:,.2f}".format(heji[name].get(year, '无')) ; set_cell_text_style(row_cells[7], 'right') # 合计金额
            # # 获取单元格中的第一个段落
            # para = merged_cell.paragraphs[0]

            # # 设置段落的对齐方式为居中
            # para.alignment = 'WD_PARAGRAPH_ALIGNMENT.CENTER'

            # # 设置段落的字体为宋体，大小为5号
            # run = para.runs[0]
            # run.font.name = '宋体'
            # run.font.size = Pt(21)  # 五号字对应21磅
            table_num += 1
                    
    doc.save(f'../2应收票据/{name}-应收票据询证函.docx')





In [23]:
name

'青岛航天半导体研究所有限公司'

In [26]:
heji['陕西铭贤电子科技有限公司']

年份
2020    2748936.0
2022    4462650.0
2023     417550.0
Name: 金额, dtype: float64

In [147]:
heji.index

MultiIndex([(    '上海瀚讯信息技术股份有限公司', 2022),
            (     '上海航天电子通讯设备研究所', 2021),
            (     '上海航天电子通讯设备研究所', 2022),
            (      '中兵通信科技股份有限公司', 2021),
            (      '中兵通信科技股份有限公司', 2022),
            (  '中国兵器工业计算机应用技术研究所', 2020),
            (  '中国兵器工业计算机应用技术研究所', 2021),
            ('中国兵器装备集团自动化研究所有限公司', 2021),
            ('中国兵器装备集团自动化研究所有限公司', 2022),
            ( '中国电子科技集团公司第三十六研究所', 2021),
            ...
            (    '陕西中科天地航空模块有限公司', 2021),
            (    '陕西中科天地航空模块有限公司', 2022),
            (      '陕西铭贤电子科技有限公司', 2020),
            (      '陕西铭贤电子科技有限公司', 2022),
            (      '陕西雷能电子科技有限公司', 2020),
            (      '陕西雷能电子科技有限公司', 2021),
            (      '陕西雷能电子科技有限公司', 2022),
            (    '青岛航天半导体研究所有限公司', 2020),
            (    '青岛航天半导体研究所有限公司', 2021),
            (    '青岛航天半导体研究所有限公司', 2022)],
           names=['被询证单位', '年份'], length=105)